In [132]:
# data from imageJ

# pixel per um in 40x image

pixel_per_um = 7.37  #all videos except 295 share this ration

pixel_per_um_295 = 4.92

fps = 6.09

In [131]:
%matplotlib qt

#imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import trackpy as tp
import pims
import pandas as pd
from pandas import DataFrame, Series  # for convenience



In [133]:
#Load files
frames = pims.open('videos/40x_final/296_sampleA.tif')
diameter_variable = 51

In [135]:
# Locate particles in first frame
f = tp.locate(frames[0], invert = False, diameter=diameter_variable,minmass = 20)
plt.figure()
tp.annotate(f, frames[0])
plt.show()

In [136]:
fig, ax = plt.subplots()
ax.hist(f['mass'], bins=20)

# Optionally, label the axes.
ax.set(xlabel='mass', ylabel='count');

avg_mass = np.average(f['mass'])


In [137]:
tp.subpx_bias(f)

c:\Python\Lib\site-packages\trackpy\plots.py:638: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  axlist = f[pos_columns].applymap(lambda x: x % 1).hist()


array([[<Axes: title={'center': 'x'}>, <Axes: title={'center': 'y'}>]],
      dtype=object)

In [139]:
f_all = tp.batch(frames[:100], diameter=diameter_variable,minmass = 20, processes=1)

Frame 99: 12 features


In [140]:
# Link particles across all the frames
t = tp.link_df(f_all, search_range=10)

tp.plot_traj(t)
plt.show()

Frame 99: 12 trajectories present.


In [144]:
t1 = tp.filter_stubs(t, 3)
# Compare the number of particles in the unfiltered and filtered data.
print('Before:', t['particle'].nunique())
print('After:', t1['particle'].nunique())

Before: 364
After: 94


In [145]:
# Plot the trajectories
tp.plot_traj(t1)
plt.show()

In [146]:
plt.figure()
tp.mass_size(t1.groupby('particle').mean()); # convenience function -- just plots size vs. mass

In [147]:
t2 = t1[(t1['size'] < 4)]

In [148]:
plt.figure()
tp.annotate(t2[t2['frame'] == 0], frames[0])

<Axes: >

In [149]:
d = tp.compute_drift(t2)

In [150]:
d.plot()
# plt.show()

<Axes: xlabel='frame'>

In [151]:
tm = tp.subtract_drift(t2.copy(), d)

In [152]:
ax = tp.plot_traj(tm)
plt.show()


In [153]:
tm = tm.rename_axis(index={'particle': 'particle_index'})
im = tp.imsd(tm, 1/pixel_per_um, fps = fps)

In [155]:
fig, ax = plt.subplots()
ax.plot(im.index, im, 'k-', alpha=0.1)  # black lines, semitransparent
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$')
ax.set_xscale('log')
ax.set_yscale('log')

In [156]:
em = tp.emsd(tm, 1/pixel_per_um,fps = fps)

In [161]:
fig, ax = plt.subplots()
ax.plot(em.index, em, 'o')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$')
#ax.set(ylim=(1e-2, 1000));

[Text(0, 0.5, '$\\langle \\Delta r^2 \\rangle$ [$\\mu$m$^2$]'),
 Text(0.5, 0, 'lag time $t$')]

In [162]:
plt.figure()
plt.ylabel(r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]')
plt.xlabel('lag time $t$');
tp.utils.fit_powerlaw(em)  # performs linear best fit in log space, plots]

# D = A/4 [um^2/s]

,n,A
msd,1.035197,2.46641
